In [7]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [156]:
data = pd.read_csv('data/combinedClimateFiresDataset_2010.csv')

### Remove extra columns from data and create labels

In [170]:
# data = data.drop(columns=['Unnamed: 0.1','Unnamed: 0_x', 'Unnamed: 0'])
# data = data.sort_values('DATE_x')
# labels = pd.DataFrame()
# labels['FIRE_SIZE'] = data['FIRE_SIZE']
# labels['FIRE_SIZE_CLASS'] = data['FIRE_SIZE_CLASS']
# labels = labels[1:]
# data = data.drop(columns=['FIRE_SIZE_CLASS','FIRE_SIZE'])

data.head()

,Unnamed: 0,Unnamed: 0_x,Unnamed: 0.1,OBJECTID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,SOURCE_REPORTING_UNIT,FIRE_NAME,FIRE_YEAR,DATE_x,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE_x,LONGITUDE_x,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_NAME,Shape,merging,Unnamed: 0_y,STATION,STATION_NAME,ELEVATION,LATITUDE_y,LONGITUDE_y,DATE_y,DLY-GRDD-BASE40,DLY-GRDD-BASE45,DLY-GRDD-BASE50,DLY-GRDD-BASE55,DLY-GRDD-BASE57,DLY-GRDD-BASE60,DLY-GRDD-BASE65,DLY-GRDD-BASE70,DLY-GRDD-BASE72,DLY-GRDD-TB4886,DLY-GRDD-TB5086,DLY-CLDD-BASE45,DLY-CLDD-BASE50,DLY-CLDD-BASE55,DLY-CLDD-BASE57,DLY-CLDD-BASE60,DLY-CLDD-NORMAL,DLY-CLDD-BASE70,DLY-CLDD-BASE72,DLY-HTDD-BASE40,DLY-HTDD-BASE45,DLY-HTDD-BASE50,DLY-HTDD-BASE55,DLY-HTDD-BASE57,DLY-HTDD-BASE60,DLY-HTDD-NORMAL,DLY-PRCP-25PCTL,DLY-SNWD-25PCTL,DLY-SNOW-25PCTL,DLY-PRCP-50PCTL,DLY-SNWD-50PCTL,DLY-SNOW-50PCTL,DLY-PRCP-75PCTL,DLY-SNWD-75PCTL,DLY-SNOW-75PCTL,MTD-PRCP-NORMAL,MTD-SNOW-NORMAL,YTD-PRCP-NORMAL,YTD-SNOW-NORMAL,DLY-PRCP-PCTALL-GE001HI,DLY-PRCP-PCTALL-GE010HI,DLY-PRCP-PCTALL-GE050HI,DLY-PRCP-PCTALL-GE100HI,DLY-SNWD-PCTALL-GE001WI,DLY-SNWD-PCTALL-GE010WI,DLY-SNWD-PCTALL-GE003WI,DLY-SNWD-PCTALL-GE005WI,DLY-SNOW-PCTALL-GE001TI,DLY-SNOW-PCTALL-GE010TI,DLY-SNOW-PCTALL-GE100TI,DLY-SNOW-PCTALL-GE030TI,DLY-SNOW-PCTALL-GE050TI,DLY-TAVG-NORMAL,DLY-DUTR-NORMAL,DLY-TMAX-NORMAL,DLY-TMIN-NORMAL,DLY-TAVG-STDDEV,DLY-DUTR-STDDEV,DLY-TMAX-STDDEV,DLY-TMIN-STDDEV,longLatOLS
4552,4552,185409,1876324,1876325,NONFED,ST-CACDF,ST/C&L,USCARRU,CARRU,HY 79 S/ VAIL LAKE RD,2009,2009-12-31,365,1440.0,2.0,Equipment Use,NaN,NaN,NaN,1.0,B,33.470,-116.953,13.0,STATE OR PRIVATE,CA,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\x14r\xa5\x9e\x05=]\...,4552,129481,GHCND:USC00044484,KELSEY 1 N CA US,609.6,38.80880,-120.82080,2010-09-29,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,0.55,-9999.0,25.27,-9999.0,91,62,27,8,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,NaN
3144,3144,145538,1219431,1219432,FED,DOI-WFMI,NPS,USCAJTP,CAJTP,ATLANTIS,2010,2010-01-01,1,1545.0,9.0,Miscellaneous,2010-01-01,1.0,2300.0,0.1,A,34.020,-116.180,3.0,NPS,CA,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\xb4\x10\xe4\xa0\x84...,3144,131035,GHCND:USC00044259,INDIO FIRE STATION CA US,-6.4,33.70860,-116.21520,2010-01-01,17,12,8,4,2,1,-7777,-7777,0,12,11,12,8,4,2,1,-7777,-7777,0,0,-7777,-7777,1,2,4,8,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,0.02,0.0,0.02,0.0,64,50,15,6,0,0,0,0,0,0,0,0,0,57.3,26.7,70.7,44.0,5.3,7.6,6.6,6.4,NaN
1393,1393,143727,1176703,1176704,FED,FS-FIRESTAT,FS,USCABDF,0512,NEW YEAR,2010,2010-01-01,1,1605.0,7.0,Arson,2010-01-01,1.0,1652.0,0.2,A,34.347,-117.442,5.0,USFS,CA,071,San Bernardino,b'\x00\x01\xad\x10\x00\x00\x94\xe4\x80]M\\]\xc...,1393,100740,GHCND:USW00003167,HAWTHORNE MUNICIPAL AIRPORT CA US,19.2,33.92278,-118.33417,2010-01-01,17,12,7,3,2,1,-7777,-7777,0,10,9,12,7,3,2,1,-7777,-7777,0,0,0,-7777,1,2,4,8,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,0.13,-9999.0,0.13,-9999.0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,57.2,17.9,66.1,48.2,4.7,7.8,7.4,4.4,NaN
1374,1374,143708,1147182,1147183,FED,FS-FIRESTAT,FS,USCASTF,0516,CEDAR,2010,2010-01-01,1,1200.0,9.0,Miscellaneous,2010-01-11,11.0,1200.0,0.1,A,38.069,-120.277,5.0,USFS,CA,109,Tuolumne,b'\x00\x01\xad\x10\x00\x00x8\x81\xe9\xb4\x11^\...,1374,87965,GHCND:USC00046252,NORTH FORK RANGER STATION CA US,806.2,37.23278,-119.50972,2010-01-01,5,2,1,-7777,-7777,0,0,0,0,5,4,2,1,-7777,-7777,0,0,0,0,1,3,7,11,13,16,21,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,0.25,-9999.0,0.25,-9999.0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,43.7,25.0,56.2,

In [158]:
# data = data.drop(columns=['DATE_x','DATE_y', 'DISCOVERY_TIME', 'CONT_DATE',
#                           'CONT_TIME','SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
#                           'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
#                           'SOURCE_REPORTING_UNIT', 'FIRE_NAME', 'OWNER_DESCR',
#                           'Shape', 'STATION', 'STATION_NAME', 'STAT_CAUSE_DESCR'])
# data = data.drop(columns=['STATE', 'COUNTY', 'FIPS_NAME'])

features = data[['DLY-GRDD-BASE40', 'DLY-GRDD-BASE45', 'DLY-GRDD-BASE50',
       'DLY-GRDD-BASE55', 'DLY-GRDD-BASE57', 'DLY-GRDD-BASE60',
       'DLY-GRDD-BASE65', 'DLY-GRDD-BASE70', 'DLY-GRDD-BASE72',
       'DLY-GRDD-TB4886', 'DLY-GRDD-TB5086', 'DLY-CLDD-BASE45',
       'DLY-CLDD-BASE50', 'DLY-CLDD-BASE55', 'DLY-CLDD-BASE57',
       'DLY-CLDD-BASE60', 'DLY-CLDD-NORMAL', 'DLY-CLDD-BASE70',
       'DLY-CLDD-BASE72', 'DLY-HTDD-BASE40', 'DLY-HTDD-BASE45',
       'DLY-HTDD-BASE50', 'DLY-HTDD-BASE55', 'DLY-HTDD-BASE57',
       'DLY-HTDD-BASE60', 'DLY-HTDD-NORMAL', 'DLY-PRCP-25PCTL',
       'DLY-SNWD-25PCTL', 'DLY-SNOW-25PCTL', 'DLY-PRCP-50PCTL',
       'DLY-SNWD-50PCTL', 'DLY-SNOW-50PCTL', 'DLY-PRCP-75PCTL',
       'DLY-SNWD-75PCTL', 'DLY-SNOW-75PCTL', 'MTD-PRCP-NORMAL',
       'MTD-SNOW-NORMAL', 'YTD-PRCP-NORMAL', 'YTD-SNOW-NORMAL',
       'DLY-PRCP-PCTALL-GE001HI', 'DLY-PRCP-PCTALL-GE010HI',
       'DLY-PRCP-PCTALL-GE050HI', 'DLY-PRCP-PCTALL-GE100HI',
       'DLY-SNWD-PCTALL-GE001WI', 'DLY-SNWD-PCTALL-GE010WI',
       'DLY-SNWD-PCTALL-GE003WI', 'DLY-SNWD-PCTALL-GE005WI',
       'DLY-SNOW-PCTALL-GE001TI', 'DLY-SNOW-PCTALL-GE010TI',
       'DLY-SNOW-PCTALL-GE100TI', 'DLY-SNOW-PCTALL-GE030TI',
       'DLY-SNOW-PCTALL-GE050TI', 'DLY-TAVG-NORMAL', 'DLY-DUTR-NORMAL',
       'DLY-TMAX-NORMAL', 'DLY-TMIN-NORMAL', 'DLY-TAVG-STDDEV',
       'DLY-DUTR-STDDEV', 'DLY-TMAX-STDDEV', 'DLY-TMIN-STDDEV',]]

In [159]:
features.columns.values
# data.dtypes
features.head()

,DLY-GRDD-BASE40,DLY-GRDD-BASE45,DLY-GRDD-BASE50,DLY-GRDD-BASE55,DLY-GRDD-BASE57,DLY-GRDD-BASE60,DLY-GRDD-BASE65,DLY-GRDD-BASE70,DLY-GRDD-BASE72,DLY-GRDD-TB4886,DLY-GRDD-TB5086,DLY-CLDD-BASE45,DLY-CLDD-BASE50,DLY-CLDD-BASE55,DLY-CLDD-BASE57,DLY-CLDD-BASE60,DLY-CLDD-NORMAL,DLY-CLDD-BASE70,DLY-CLDD-BASE72,DLY-HTDD-BASE40,DLY-HTDD-BASE45,DLY-HTDD-BASE50,DLY-HTDD-BASE55,DLY-HTDD-BASE57,DLY-HTDD-BASE60,DLY-HTDD-NORMAL,DLY-PRCP-25PCTL,DLY-SNWD-25PCTL,DLY-SNOW-25PCTL,DLY-PRCP-50PCTL,DLY-SNWD-50PCTL,DLY-SNOW-50PCTL,DLY-PRCP-75PCTL,DLY-SNWD-75PCTL,DLY-SNOW-75PCTL,MTD-PRCP-NORMAL,MTD-SNOW-NORMAL,YTD-PRCP-NORMAL,YTD-SNOW-NORMAL,DLY-PRCP-PCTALL-GE001HI,DLY-PRCP-PCTALL-GE010HI,DLY-PRCP-PCTALL-GE050HI,DLY-PRCP-PCTALL-GE100HI,DLY-SNWD-PCTALL-GE001WI,DLY-SNWD-PCTALL-GE010WI,DLY-SNWD-PCTALL-GE003WI,DLY-SNWD-PCTALL-GE005WI,DLY-SNOW-PCTALL-GE001TI,DLY-SNOW-PCTALL-GE010TI,DLY-SNOW-PCTALL-GE100TI,DLY-SNOW-PCTALL-GE030TI,DLY-SNOW-PCTALL-GE050TI,DLY-TAVG-NORMAL,DLY-DUTR-NORMAL,DLY-TMAX-NORMAL,DLY-TMIN-NORMAL,DLY-TAVG-STDDEV,DLY-DUTR-STDDEV,DLY-TMAX-STDDEV,DLY-TMIN-STDDEV
4552,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,0.55,-9999.0,25.27,-9999.0,91,62,27,8,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3144,17,12,8,4,2,1,-7777,-7777,0,12,11,12,8,4,2,1,-7777,-7777,0,0,-7777,-7777,1,2,4,8,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,0.02,0.0,0.02,0.0,64,50,15,6,0,0,0,0,0,0,0,0,0,57.3,26.7,70.7,44.0,5.3,7.6,6.6,6.4
1393,17,12,7,3,2,1,-7777,-7777,0,10,9,12,7,3,2,1,-7777,-7777,0,0,0,-7777,1,2,4,8,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,0.13,-9999.0,0.13,-9999.0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,57.2,17.9,66.1,48.2,4.7,7.8,7.4,4.4
1374,5,2,1,-7777,-7777,0,0,0,0,5,4,2,1,-7777,-7777,0,0,0,0,1,3,7,11,13,16,21,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,0.25,-9999.0,0.25,-9999.0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,43.7,25.0,56.2,31.2,-9999.0,-9999.0,-9999.0,-9999.0
245,4,1,-7777,-7777,0,0,0,0,0,2,1,1,-7777,-7777,0,0,0,0,0,-7777,3,6,11,13,16,21,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,0.23,-9999.0,0.23,-9999.0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,43.7,15.4,51.4,36.0,5.0,8.1,5.8,7.0


In [160]:
# # Replace negative values by na, add date, and sort by date
features[features < 0] = None



/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:3163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)


In [120]:
missing_percentage = []
for column in features.columns:
    missing_percentage.append(str(int(features[column].isna().sum()/len(features[column])*100)) + "% missing in "+ column)
missing_percentage.sort()
missing_percentage

['18% missing in DLY-CLDD-BASE45',
 '18% missing in DLY-CLDD-BASE50',
 '18% missing in DLY-DUTR-NORMAL',
 '18% missing in DLY-GRDD-BASE40',
 '18% missing in DLY-GRDD-BASE45',
 '18% missing in DLY-GRDD-BASE50',
 '18% missing in DLY-GRDD-TB4886',
 '18% missing in DLY-GRDD-TB5086',
 '18% missing in DLY-TAVG-NORMAL',
 '18% missing in DLY-TMAX-NORMAL',
 '18% missing in DLY-TMIN-NORMAL',
 '2% missing in MTD-PRCP-NORMAL',
 '2% missing in YTD-PRCP-NORMAL',
 '20% missing in DLY-CLDD-BASE55',
 '20% missing in DLY-GRDD-BASE55',
 '21% missing in DLY-CLDD-BASE57',
 '21% missing in DLY-GRDD-BASE57',
 '22% missing in DLY-PRCP-PCTALL-GE001HI',
 '24% missing in DLY-CLDD-BASE60',
 '24% missing in DLY-GRDD-BASE60']

In [161]:
features = features[['DLY-CLDD-BASE45',
 'DLY-CLDD-BASE50',
 'DLY-DUTR-NORMAL',
 'DLY-GRDD-BASE40',
 'DLY-GRDD-BASE45',
 'DLY-GRDD-BASE50',
 'DLY-GRDD-TB4886',
 'DLY-GRDD-TB5086',
 'DLY-TAVG-NORMAL',
 'DLY-TMAX-NORMAL',
 'DLY-TMIN-NORMAL',
 'MTD-PRCP-NORMAL',
 'YTD-PRCP-NORMAL',
 'DLY-CLDD-BASE55',
 'DLY-GRDD-BASE55',
 'DLY-CLDD-BASE57',
 'DLY-GRDD-BASE57',
 'DLY-PRCP-PCTALL-GE001HI',
 'DLY-CLDD-BASE60',
 'DLY-GRDD-BASE60',]]





In [164]:
# FILL NA VALUES WITH FFILL
# features.fillna(method='ffill', inplace=True)
# features = features[1:]

features.head()

,DLY-CLDD-BASE45,DLY-CLDD-BASE50,DLY-DUTR-NORMAL,DLY-GRDD-BASE40,DLY-GRDD-BASE45,DLY-GRDD-BASE50,DLY-GRDD-TB4886,DLY-GRDD-TB5086,DLY-TAVG-NORMAL,DLY-TMAX-NORMAL,DLY-TMIN-NORMAL,MTD-PRCP-NORMAL,YTD-PRCP-NORMAL,DLY-CLDD-BASE55,DLY-GRDD-BASE55,DLY-CLDD-BASE57,DLY-GRDD-BASE57,DLY-PRCP-PCTALL-GE001HI,DLY-CLDD-BASE60,DLY-GRDD-BASE60
3144,12.0,8.0,26.7,17.0,12.0,8.0,12.0,11.0,57.3,70.7,44.0,0.02,0.02,4.0,4.0,2.0,2.0,64.0,1.0,1.0
1393,12.0,7.0,17.9,17.0,12.0,7.0,10.0,9.0,57.2,66.1,48.2,0.13,0.13,3.0,3.0,2.0,2.0,64.0,1.0,1.0
1374,2.0,1.0,25.0,5.0,2.0,1.0,5.0,4.0,43.7,56.2,31.2,0.25,0.25,3.0,3.0,2.0,2.0,64.0,0.0,0.0
245,1.0,1.0,15.4,4.0,1.0,1.0,2.0,1.0,43.7,51.4,36.0,0.23,0.23,3.0,3.0,0.0,0.0,64.0,0.0,0.0
964,1.0,1.0,15.4,4.0,1.0,1.0,2.0,1.0,43.7,51.4,36.0,0.05,0.05,3.0,3.0,0.0,0.0,108.0,0.0,0.0


## Predicting FIRE_SIZE_CLASS
### TO-DO
* Create dummy variables for categorical data

In [150]:
from sklearn.feature_selection import SelectKBest, chi2, RFE, VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from itertools import compress


In [176]:
new_features = SelectKBest(chi2, k=10).fit(features, labels.FIRE_SIZE_CLASS)
model = RandomForestClassifier()
rfe = RFE(model, 10)

In [177]:
rfe.fit(features, labels.FIRE_SIZE_CLASS)

RFE(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
  n_features_to_select=10, step=1, verbose=0)

In [178]:
list(compress(features.columns.values, new_features.get_support()))

['DLY-CLDD-BASE45',
 'DLY-CLDD-BASE50',
 'DLY-GRDD-BASE40',
 'DLY-GRDD-BASE45',
 'DLY-GRDD-BASE50',
 'YTD-PRCP-NORMAL',
 'DLY-CLDD-BASE55',
 'DLY-GRDD-BASE55',
 'DLY-GRDD-BASE57',
 'DLY-PRCP-PCTALL-GE001HI']

In [179]:
list(compress(features.columns.values, rfe.get_support()))

['DLY-DUTR-NORMAL',
 'DLY-GRDD-TB5086',
 'DLY-TAVG-NORMAL',
 'DLY-TMAX-NORMAL',
 'DLY-TMIN-NORMAL',
 'MTD-PRCP-NORMAL',
 'YTD-PRCP-NORMAL',
 'DLY-CLDD-BASE55',
 'DLY-PRCP-PCTALL-GE001HI',
 'DLY-GRDD-BASE60']

In [167]:
features.shape

(5775, 20)

In [171]:
labels.shape

(5775, 2)